In [106]:
import copy
import itertools
import json
import nltk
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from categorizer import *
from clean import  *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [107]:
# Tokenize each message a customer sent
import os
import re
from nltk.tokenize import sent_tokenize

sentence_customer = []

for file in os.listdir("question"):
    with open(f"question/{file}", "r") as f:
        sentences = sent_tokenize(f.read())
        print(sentences)
        sentence_customer.append(sentences)

["Hello, i was interesting in the “ STUDIO D'ARTISAN 14OZ.", '"MOUNT FUJI" SELVEDGE JEANS (RELAX TAPERED)”.', 'My question is that I’ve never bought a raw pair of jeans before and going based on the measurements of the one wash, my waist would be in the 31.5-32 inch size.', 'Knowing that, would the 31 inch raw size shrink to the one wash measurements?', 'And how much will it stretch back out after wear?', 'Please let me know, thank you!']
['ORDER #59667 -- I ordered the shirt in a size 4 but I actually think a 5 would be better.', 'Could you change the order to a 5?', 'Thank you.', 'Julian Aiken']
["hi i don't see the sizing for this pair of momotaro?", 'my waist size is 32.5 inches in diameter, what would be the correct sizing if i want my jeans to fit slightly loose on me.']
['Any chance PBJ will launch type III denim jackets this season?']
['hello, i am looking at ordering the Fullcount 1108EC ecru selvedge jeans however because this is my first time buying this brand if the size do

In [108]:
token = []
for sentence in sentence_customer:
    token.append(clean(sentence))

In [109]:
response = []
with open("categorize.text", "r") as file:
    for f in file.readlines():
        response.append(f.lower().replace("\n", ""))

In [110]:
new_token = []
for t, r in zip(token, response):
    tok = []
    n = 0

    if len(t) > 19:
        while n < len(t) - 20:
            tok.append(t[n:n+20])
            n += 1

    else:
        tok.append(t)

    tok.append([r])
    new_token.append(tok)


In [111]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
for text in new_token:
    tokenizer.fit_on_texts(text)

In [112]:
X = []
y = []
for text in new_token:
    X.append(tokenizer.texts_to_sequences(text[:-1]))
    i = 0
    while i in range(len(text[:-1])):
        y.append(tokenizer.texts_to_sequences(text[-1]))
        i += 1

In [113]:
X = [inner_list for middle_list in X for inner_list in middle_list]
y = [inner_list for middle_list in y for inner_list in middle_list]

In [114]:
vocab_size = len(tokenizer.word_index) + 1
X = np.array(pad_sequences(X, truncating = "post", maxlen = len(max(X, key = len))))
y = np.array(pad_sequences(y, truncating = "post", maxlen = len(max(y, key = len))))

In [115]:
y = tf.keras.utils.to_categorical(y, num_classes = vocab_size)

In [116]:
input_length = X.shape[1]
output_length = y.shape[1]

In [117]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length = input_length))
model.add(LSTM(100, return_sequences = True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(output_length * vocab_size, activation='softmax'))
model.add(tf.keras.layers.Reshape((output_length, vocab_size)))
print(model.summary())


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 20, 100)           179000    
                                                                 
 lstm_16 (LSTM)              (None, 20, 100)           80400     
                                                                 
 lstm_17 (LSTM)              (None, 100)               80400     
                                                                 
 dense_16 (Dense)            (None, 100)               10100     
                                                                 
 dense_17 (Dense)            (None, 3580)              361580    
                                                                 
 reshape_8 (Reshape)         (None, 2, 1790)           0         
                                                                 
Total params: 711480 (2.71 MB)
Trainable params: 71148

In [118]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.001)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
model.fit(X, y, batch_size = 256, epochs = 250)

Epoch 1/250
19/19 [==============================] - 3s 68ms/step - loss: 6.6149 - accuracy: 0.4537
Epoch 2/250
19/19 [==============================] - 1s 70ms/step - loss: 1.7806 - accuracy: 0.4614
Epoch 3/250
19/19 [==============================] - 1s 75ms/step - loss: 1.5045 - accuracy: 0.4795
Epoch 4/250
19/19 [==============================] - 1s 72ms/step - loss: 1.4964 - accuracy: 0.4795
Epoch 5/250
19/19 [==============================] - 1s 72ms/step - loss: 1.4929 - accuracy: 0.4795
Epoch 6/250
19/19 [==============================] - 1s 73ms/step - loss: 1.4951 - accuracy: 0.4795
Epoch 7/250
19/19 [==============================] - 1s 77ms/step - loss: 1.4950 - accuracy: 0.4795
Epoch 8/250
19/19 [==============================] - 1s 76ms/step - loss: 1.4986 - accuracy: 0.4795
Epoch 9/250
19/19 [==============================] - 1s 77ms/step - loss: 1.4917 - accuracy: 0.4795
Epoch 10/250
19/19 [==============================] - 1s 76ms/step - loss: 1.4919 - accuracy: 0.4795

In [65]:
model.save('model/model1.keras')
pickle.dump(tokenizer, open('token/tokenizer.pkl', 'wb'))

In [100]:
test = tokenizer.texts_to_sequences(token[1236:1237])

test = pad_sequences(test, maxlen = input_length, truncating = 'post')

result = model.predict(test, verbose = 1)

word_indices = [np.argmax(r, axis = 1) for r in result]

final = tokenizer.sequences_to_texts(word_indices)

print(final)

1/1 [==============================] - 0s 54ms/step
['restock']


In [101]:
token[1236:1237]

[['hi',
  'how',
  'much',
  'are',
  'the',
  'new',
  '25oz',
  'samurais',
  'expected',
  'to',
  'stretch',
  'in',
  'the',
  'waist']]